# Portfolio Theory

## Overview
Modern Portfolio Theory (MPT), introduced by Harry Markowitz in 1952, provides a framework for constructing portfolios that maximize expected return for a given level of risk, or minimize risk for a given return.

## Portfolio Return and Risk

### Expected Portfolio Return
$$E(R_p) = \sum_{i=1}^{n} w_i E(R_i)$$

### Portfolio Variance (2 assets)
$$\sigma_p^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\sigma_1\sigma_2\rho_{12}$$

### Portfolio Variance (n assets)
$$\sigma_p^2 = \sum_{i=1}^{n}\sum_{j=1}^{n} w_i w_j \sigma_{ij} = \mathbf{w}^T \mathbf{\Sigma} \mathbf{w}$$

Where $\mathbf{\Sigma}$ is the covariance matrix.

## The Power of Diversification

When assets are not perfectly correlated ($\rho < 1$), diversification reduces portfolio risk without sacrificing expected return.

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Two-asset portfolio with different correlations
w1 = np.linspace(0, 1, 100)
w2 = 1 - w1

r1, r2 = 0.10, 0.15  # Expected returns
s1, s2 = 0.12, 0.25  # Standard deviations

correlations = [1.0, 0.5, 0.0, -0.5, -1.0]
colors = ['#e74c3c', '#f39c12', '#2ecc71', '#3498db', '#9b59b6']

fig = go.Figure()

for rho, color in zip(correlations, colors):
    port_return = w1 * r1 + w2 * r2
    port_vol = np.sqrt(w1**2 * s1**2 + w2**2 * s2**2 + 2*w1*w2*s1*s2*rho)
    
    fig.add_trace(go.Scatter(
        x=port_vol * 100, y=port_return * 100,
        mode='lines', name=f'ρ = {rho}',
        line=dict(color=color, width=2)
    ))

# Mark individual assets
fig.add_trace(go.Scatter(
    x=[s1*100, s2*100], y=[r1*100, r2*100],
    mode='markers+text', name='Assets',
    marker=dict(size=12, color='black'),
    text=['Asset 1', 'Asset 2'], textposition='top center'
))

fig.update_layout(
    title='Diversification Effect: Portfolio Risk at Different Correlations',
    xaxis_title='Portfolio Volatility (%)',
    yaxis_title='Expected Return (%)',
    template='plotly_white'
)
fig

## The Efficient Frontier

The efficient frontier is the set of optimal portfolios offering:
- Maximum expected return for a given level of risk, OR
- Minimum risk for a given level of expected return

Portfolios **below** the frontier are suboptimal (dominated by efficient portfolios).

### Minimum Variance Portfolio (MVP)
The portfolio with the lowest possible volatility:

$$\mathbf{w}_{MVP} = \frac{\mathbf{\Sigma}^{-1} \mathbf{1}}{\mathbf{1}^T \mathbf{\Sigma}^{-1} \mathbf{1}}$$

In [ ]:
np.random.seed(42)

# 5 assets
n_assets = 5
expected_returns = np.array([0.08, 0.10, 0.12, 0.14, 0.16])
volatilities = np.array([0.15, 0.18, 0.22, 0.25, 0.30])

# Generate correlation matrix
corr_matrix = np.array([
    [1.0, 0.3, 0.2, 0.1, 0.05],
    [0.3, 1.0, 0.4, 0.2, 0.1],
    [0.2, 0.4, 1.0, 0.5, 0.3],
    [0.1, 0.2, 0.5, 1.0, 0.6],
    [0.05, 0.1, 0.3, 0.6, 1.0]
])

# Covariance matrix
cov_matrix = np.outer(volatilities, volatilities) * corr_matrix

# Generate random portfolios
n_portfolios = 5000
results = np.zeros((n_portfolios, 3))
weights_record = []

for i in range(n_portfolios):
    weights = np.random.random(n_assets)
    weights /= weights.sum()
    
    port_return = np.dot(weights, expected_returns)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = port_return / port_vol
    
    results[i] = [port_vol, port_return, sharpe]
    weights_record.append(weights)

# Find optimal portfolios
max_sharpe_idx = results[:, 2].argmax()
min_vol_idx = results[:, 0].argmin()

fig = go.Figure()

# All portfolios
fig.add_trace(go.Scatter(
    x=results[:, 0] * 100, y=results[:, 1] * 100,
    mode='markers',
    marker=dict(size=4, color=results[:, 2], colorscale='Viridis', 
                colorbar=dict(title='Sharpe Ratio')),
    name='Portfolios'
))

# Optimal portfolios
fig.add_trace(go.Scatter(
    x=[results[max_sharpe_idx, 0] * 100], y=[results[max_sharpe_idx, 1] * 100],
    mode='markers', name='Max Sharpe',
    marker=dict(size=15, color='red', symbol='star')
))

fig.add_trace(go.Scatter(
    x=[results[min_vol_idx, 0] * 100], y=[results[min_vol_idx, 1] * 100],
    mode='markers', name='Min Volatility',
    marker=dict(size=15, color='blue', symbol='diamond')
))

# Individual assets
fig.add_trace(go.Scatter(
    x=volatilities * 100, y=expected_returns * 100,
    mode='markers+text', name='Individual Assets',
    marker=dict(size=10, color='black'),
    text=[f'Asset {i+1}' for i in range(n_assets)], textposition='top center'
))

fig.update_layout(
    title='Efficient Frontier with Random Portfolios',
    xaxis_title='Volatility (%)', yaxis_title='Expected Return (%)',
    template='plotly_white'
)
fig.show()

print(f"Max Sharpe Portfolio: Return={results[max_sharpe_idx,1]*100:.2f}%, Vol={results[max_sharpe_idx,0]*100:.2f}%, Sharpe={results[max_sharpe_idx,2]:.2f}")
print(f"Min Vol Portfolio:    Return={results[min_vol_idx,1]*100:.2f}%, Vol={results[min_vol_idx,0]*100:.2f}%")

## Capital Market Line (CML)

When a risk-free asset is available, the optimal portfolio is found where the Capital Market Line is tangent to the efficient frontier:

$$E(R_p) = R_f + \frac{E(R_m) - R_f}{\sigma_m} \sigma_p$$

The tangent portfolio is the **Market Portfolio** in equilibrium.

## Capital Asset Pricing Model (CAPM)

$$E(R_i) = R_f + \beta_i (E(R_m) - R_f)$$

Where:
- $\beta_i = \frac{\text{Cov}(R_i, R_m)}{\text{Var}(R_m)}$: Systematic risk
- $E(R_m) - R_f$: Market risk premium

**Key Insight:** Only systematic (market) risk is rewarded; idiosyncratic risk can be diversified away.

In [ ]:
# Security Market Line (SML)
rf = 0.03  # Risk-free rate
market_return = 0.10
market_premium = market_return - rf

betas = np.linspace(0, 2, 100)
expected_returns_sml = rf + betas * market_premium

# Sample stocks
stocks = [
    {'name': 'Utilities', 'beta': 0.4, 'actual': 0.055},
    {'name': 'Consumer', 'beta': 0.8, 'actual': 0.085},
    {'name': 'Market', 'beta': 1.0, 'actual': 0.10},
    {'name': 'Tech', 'beta': 1.4, 'actual': 0.13},
    {'name': 'Undervalued', 'beta': 0.9, 'actual': 0.11},
    {'name': 'Overvalued', 'beta': 1.2, 'actual': 0.08},
]

fig = go.Figure()

# SML line
fig.add_trace(go.Scatter(
    x=betas, y=expected_returns_sml * 100,
    mode='lines', name='Security Market Line',
    line=dict(color='#3498db', width=2)
))

# Stocks
for stock in stocks:
    expected = rf + stock['beta'] * market_premium
    color = '#2ecc71' if stock['actual'] > expected else '#e74c3c'
    
    fig.add_trace(go.Scatter(
        x=[stock['beta']], y=[stock['actual'] * 100],
        mode='markers+text', name=stock['name'],
        marker=dict(size=12, color=color),
        text=[stock['name']], textposition='top center'
    ))

fig.update_layout(
    title='Security Market Line (CAPM)',
    xaxis_title='Beta (β)', yaxis_title='Expected Return (%)',
    template='plotly_white'
)
fig.add_annotation(x=1.5, y=7, text='Above SML = Undervalued (Buy)', showarrow=False, font=dict(color='#2ecc71'))
fig.add_annotation(x=1.5, y=6, text='Below SML = Overvalued (Sell)', showarrow=False, font=dict(color='#e74c3c'))
fig

## Factor Models

### Fama-French Three-Factor Model
$$R_i - R_f = \alpha_i + \beta_{m}(R_m - R_f) + \beta_{smb} \cdot SMB + \beta_{hml} \cdot HML + \epsilon_i$$

- **SMB (Small Minus Big)**: Size factor
- **HML (High Minus Low)**: Value factor (Book-to-Market)

### Additional Factors
- **Momentum (MOM)**: Winners continue winning
- **Quality (QMJ)**: Profitable, growing, safe companies
- **Low Volatility**: Low-risk stocks outperform

## Key Takeaways

1. **Diversification is free** — Reduces risk without sacrificing return (when ρ < 1)
2. **The efficient frontier is the goal** — Only efficient portfolios are optimal
3. **Risk-free asset changes everything** — Enables borrowing/lending on CML
4. **Beta measures systematic risk** — The only risk rewarded by the market
5. **CAPM is a starting point** — Multi-factor models explain more variation